# Zoom Polarization Analysis

## Introduction

In [ ]:
import scipp as sc
from ess import sans
from ess import polarization as pol
from ess import isissans as isis
from ess.sans.types import *

In [ ]:
sans_workflow = isis.zoom.ZoomWorkflow()
sans_workflow.set_param_series(PixelMaskFilename, [])

In [ ]:
from pathlib import Path

base = Path(
    '~/instruments/polarization/Polarized_data_ZOOM/full_polarized_TOF-SANS_example_data_on_glassy_carbon'
)
data_folder = base / 'Long_3He_run'
data_folder = base / 'GC/3He_GC'
# Runs with analyzer at 4 different times
cell_runs = [data_folder / f'ZOOM00022{run}.nxs' for run in [710, 712, 714, 716]]
empty_run = data_folder / 'ZOOM00034787.nxs'
cell_runs
depolarized_run = data_folder / 'ZOOM00022718.nxs'

In [ ]:
import mantid.api as _mantid_api
from mantid import simpleapi as _mantid_simpleapi
from typing import Optional
from ess.isissans.mantidio import Period, DataWorkspace


sample_run_type = TransmissionRun[SampleRun]


def load_histogrammed_run(
    filename: Filename[sample_run_type], period: Optional[Period]
) -> DataWorkspace[sample_run_type]:
    """Load a non-event-data ISIS file"""
    loaded = _mantid_simpleapi.Load(Filename=str(filename), StoreInADS=False)
    if isinstance(loaded, _mantid_api.Workspace):
        # A single workspace
        data_ws = loaded
        if isinstance(data_ws, _mantid_api.WorkspaceGroup):
            if period is None:
                raise ValueError(
                    f'Needs {Period} to be set to know what '
                    'section of the event data to load'
                )
            data_ws = data_ws.getItem(period)
    else:
        # Separate data and monitor workspaces
        data_ws = loaded.OutputWorkspace
        if isinstance(data_ws, _mantid_api.WorkspaceGroup):
            if period is None:
                raise ValueError(
                    f'Needs {Period} to be set to know what '
                    'section of the event data to load'
                )
            data_ws = data_ws.getItem(period)
            data_ws.setMonitorWorkspace(loaded.MonitorWorkspace.getItem(period))
        else:
            data_ws.setMonitorWorkspace(loaded.MonitorWorkspace)
    return DataWorkspace[sample_run_type](data_ws)


def get_incident(
    dg: isis.data.LoadedFileContents[sample_run_type],
) -> RawMonitor[sample_run_type, Incident]:
    """Extract indcident monitor from ZOOM direct-beam run"""
    return RawMonitor[sample_run_type, Incident](dg['data']['spectrum', 2].copy())


def get_transmission(
    dg: isis.data.LoadedFileContents[sample_run_type],
) -> RawMonitor[sample_run_type, Transmission]:
    """Extract transmission monitor from ZOOM direct-beam run"""
    monitor = dg['data']['spectrum', 4].copy()
    monitor.coords['datetime'] = get_time(dg)
    return RawMonitor[sample_run_type, Transmission](monitor)


from typing import NewType


RunTime = NewType('RunTime', sc.Variable)


def get_time(dg: isis.data.LoadedFileContents[sample_run_type]) -> RunTime:
    start = sc.datetime(dg['run_start'].value)
    end = sc.datetime(dg['run_end'].value)
    delta = end - start
    return RunTime(start + delta // 2)

In [ ]:
sans_workflow[Filename[SampleRun]] = str(cell_runs[0])
sans_workflow[Filename[TransmissionRun[SampleRun]]] = str(cell_runs[0])
sans_workflow[Filename[EmptyBeamRun]] = str(empty_run)
sans_workflow[NeXusMonitorName[Incident]] = 'monitor3'
sans_workflow[NeXusMonitorName[Transmission]] = 'monitor5'
sans_workflow.insert(load_histogrammed_run)
sans_workflow.insert(get_incident)
sans_workflow.insert(get_transmission)
sans_workflow.insert(get_time)
sans_workflow[WavelengthBins] = sc.geomspace(
    'wavelength', start=1.75, stop=16.5, num=141, unit='angstrom'
)
sans_workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
sans_workflow[isis.mantidio.Period] = 0
dg = sans_workflow.compute(isis.mantidio.LoadedFileContents[TransmissionRun[SampleRun]])
dg['data'].plot()

In [ ]:
sans_workflow.compute(RunTime)

In [ ]:
sans_workflow.visualize(TransmissionFraction[SampleRun], graph_attr={'rankdir': 'LR'})
transmission_fraction = sans_workflow.compute(TransmissionFraction[SampleRun])
transmission_fraction.plot()
# display(sans_workflow.compute(TofMonitor[SampleRun, Incident]).plot())
# display(sans_workflow.compute(TofMonitor[SampleRun, Transmission]).plot())
# display(sans_workflow.compute(TofMonitor[EmptyBeamRun, Incident]).hist().plot())
# display(sans_workflow.compute(TofMonitor[EmptyBeamRun, Transmission]).hist().plot())

In [ ]:
cell_runs

In [ ]:
sans_workflow[Filename[TransmissionRun[SampleRun]]] = str(depolarized_run)
depolarized_transmission_fraction = sans_workflow.compute(
    TransmissionFraction[SampleRun]
)

In [ ]:
depolarized_transmission_fraction.plot()

In [ ]:
transmission_fractions = []
for cell_run in cell_runs:
    sans_workflow[Filename[SampleRun]] = str(cell_run)
    sans_workflow[Filename[TransmissionRun[SampleRun]]] = str(cell_run)
    transmission_fractions.append(
        sans_workflow.compute(TransmissionFraction[SampleRun])
    )

In [ ]:
transmission_fraction = sc.concat(transmission_fractions, 'time')
datetime = transmission_fraction.coords['datetime']
transmission_fraction.coords['time'] = datetime - datetime.min()
transmission_fraction.variances = None
wav_min = 2.2 * sc.Unit('angstrom')
wav_max = 2.8 * sc.Unit('angstrom')
transmission_fraction = transmission_fraction['wavelength', wav_min:wav_max]
depolarized_transmission_fraction = depolarized_transmission_fraction[
    'wavelength', wav_min:wav_max
].copy()
depolarized_transmission_fraction.variances = None
transmission_fraction.plot()

In [ ]:
depolarized_transmission_fraction.plot()

In [ ]:
# Sanity check: Where can cosh yield values that can be fitted?
transmission_empty_glass = 0.9 * sc.Unit('dimensionless')
wavelength = sc.midpoints(transmission_fraction.coords['wavelength'])
opacity0 = 0.8797823016804095 * sc.Unit('1/angstrom')
(
    sc.acosh(
        transmission_fraction * sc.exp(opacity0 * wavelength) / transmission_empty_glass
    )
    / (opacity0 * wavelength)
).plot()

In [ ]:
pol_workflow = pol.he3.He3CellWorkflow(in_situ=False)
pol_workflow[
    pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Polarized]
] = transmission_fraction
pol_workflow[
    pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Depolarized]
] = depolarized_transmission_fraction
pol_workflow[pol.he3.He3CellLength[pol.Analyzer]] = 0.1 * sc.Unit('m')
pol_workflow[pol.he3.He3CellPressure[pol.Analyzer]] = 1.0 * sc.Unit('bar')
pol_workflow[pol.he3.He3CellTemperature[pol.Analyzer]] = 300.0 * sc.Unit('K')
pol_workflow[pol.he3.He3TransmissionEmptyGlass[pol.Analyzer]] = transmission_empty_glass
pol_workflow.visualize(pol.he3.He3TransmissionFunction[pol.Analyzer])
func = pol_workflow.compute(pol.he3.He3TransmissionFunction[pol.Analyzer])

In [ ]:
wavelength = sc.linspace('wavelength', start=1.75, stop=16.5, num=141, unit='angstrom')
time = sc.linspace('time', start=0, stop=10000, num=100, unit='s')
func.opacity_function(wavelength=wavelength).plot()
display(func.polarization_function(time=time).plot())
display(func(wavelength=wavelength, time=time, plus_minus='plus').plot())
display(func(wavelength=wavelength, time=time, plus_minus='minus').plot())

In [ ]:
func.opacity_function.opacity0

In [ ]:
func.polarization_function.C

In [ ]:
func.polarization_function.T1